In [5]:
!pip install pytest

**Import Libraries**

In [6]:
import numpy as np
import pytest
from numpy.testing import assert_almost_equal, assert_allclose

**State Preparation Function**

In [7]:
def _normalize_amplitudes(amplitudes_array):
    """
    Internal helper function to normalize a quantum state vector.

    Args:
        amplitudes_array (np.ndarray): A NumPy array of complex amplitudes.

    Returns:
        np.ndarray: A normalized NumPy array.

    Raises:
        ValueError: If the input amplitudes form a zero vector.
    """
    # Calculate the L2 norm (magnitude) of the vector
    norm = np.linalg.norm(amplitudes_array)

    # Use np.isclose for a robust check against zero
    if np.isclose(norm, 0.0):
        raise ValueError("Input amplitudes form a zero vector, which cannot be normalized.")

    # Divide each amplitude by the norm
    return amplitudes_array / norm

def prepare_two_qubit_state(amplitudes):
    """
    Prepares a normalized two-qubit quantum state vector.

    Parameters:
    -----------
    amplitudes : array-like
        A list or array of 4 complex numbers [a0, a1, a2, a3] for the state:
        |ψ⟩ = a0|00⟩ + a1|01⟩ + a2|10⟩ + a3|11⟩

    Returns:
    --------
    np.ndarray
        A normalized state vector of shape (4,).

    Raises:
    -------
    ValueError
        If the input does not have exactly 4 amplitudes.

    Example:
    --------
    >>> # Prepare a Bell state from an unnormalized input
    >>> state = prepare_two_qubit_state([1, 0, 0, 1])
    >>> print(state)
    [0.70710678+0.j 0.          +0.j 0.          +0.j 0.70710678+0.j]
    """
    # Convert to a NumPy array and validate shape
    # We use np.asarray to avoid copying if it's already an array
    amplitudes_array = np.asarray(amplitudes, dtype=np.complex128)

    # Use the more idiomatic .shape check
    if amplitudes_array.shape != (4,):
        raise ValueError(f"Expected 4 amplitudes for a two-qubit state, but got {amplitudes_array.size}.")

    # Delegate the actual math to our clean helper function
    return _normalize_amplitudes(amplitudes_array)

def compute_measurement_probabilities(state_vector):
    """
    (Helper Function) Computes the measurement probabilities for each basis state.

    The probability of measuring a basis state |i⟩ is |a_i|².

    Parameters:
    -----------
    state_vector : np.ndarray
        A normalized quantum state vector.

    Returns:
    --------
    np.ndarray
        An array of real-valued probabilities, where each element is |a_i|².
    """
    # np.abs(x) calculates the magnitude (modulus) of a complex number
    # **2 squares it. This is the correct way to get |a_i|².
    return np.abs(state_vector)**2


# --- STRETCH GOAL ---

def prepare_n_qubit_state(amplitudes, num_qubits):
    """
    (Stretch Goal) Prepares a normalized n-qubit quantum state vector.

    Parameters:
    -----------
    amplitudes : array-like
        A list or array of 2^n complex amplitudes.
    num_qubits : int
        The number of qubits (e.g., 3 for a 3-qubit state).

    Returns:
    --------
    np.ndarray
        A normalized state vector of shape (2^n,).

    Raises:
    -------
    ValueError
        If the number of amplitudes does not match 2^num_qubits.
    """
    expected_size = 2**num_qubits

    # Convert and validate
    amplitudes_array = np.asarray(amplitudes, dtype=np.complex128)

    if amplitudes_array.size != expected_size:
        raise ValueError(f"Expected {expected_size} amplitudes for a {num_qubits}-qubit state, "
                         f"but got {amplitudes_array.size}.")

    # Reshape to a 1D vector (in case a multi-dim array was passed)
    # and then normalize.
    return _normalize_amplitudes(amplitudes_array.reshape(-1))

**Unit Test Functions**

In [8]:
# ============================================
#             UNIT TESTS
# ============================================

def test_normalization_is_enforced():
    """Tests that an unnormalized vector [1, 1, 1, 1] is correctly normalized."""
    unnormalized_amps = [1, 1, 1, 1]
    state = prepare_two_qubit_state(unnormalized_amps)
    norm = np.linalg.norm(state)
    assert_almost_equal(norm, 1.0)
    expected_state = [0.5, 0.5, 0.5, 0.5]
    assert_allclose(state, expected_state)
    print("✓ Test 1 passed: Normalization enforced")

def test_already_normalized_vector():
    """Tests that an already normalized Bell state is unchanged."""
    val = 1 / np.sqrt(2)
    normalized_amps = [val, 0, 0, val]
    state = prepare_two_qubit_state(normalized_amps)
    assert_almost_equal(np.linalg.norm(state), 1.0)
    assert_allclose(state, normalized_amps)
    print("✓ Test 2 passed: Already normalized vector handled")

def test_output_dimension_is_correct():
    """Tests that the output vector has the correct dimension (4)."""
    state = prepare_two_qubit_state([1, 0, 0, 0])
    assert state.shape == (4,)
    print("✓ Test 3 passed: Correct output dimension (4,)")

def test_invalid_dimension_raises_error():
    """Tests that ValueError is raised for incorrect number of amplitudes."""
    with pytest.raises(ValueError, match="Expected 4 amplitudes"):
        prepare_two_qubit_state([1, 0, 0]) # Only 3 amplitudes
    print("✓ Test 4 passed: Error raised for invalid dimension")

def test_zero_vector_raises_error():
    """Tests that ValueError is raised for a zero vector (which can't be normalized)."""
    with pytest.raises(ValueError, match="zero vector"):
        prepare_two_qubit_state([0, 0, 0, 0])
    print("✓ Test 5 passed: Zero vector error handling")

def test_measurement_probabilities():
    """Tests our new helper function for computing probabilities."""
    state = prepare_two_qubit_state([1, 1j, 0, 0]) # Norm is sqrt(1^2 + |1j|^2) = sqrt(2)
    # Expected state is [1/sqrt(2), 1j/sqrt(2), 0, 0]
    # Expected probs are [0.5, 0.5, 0, 0]
    probs = compute_measurement_probabilities(state)
    assert_allclose(probs, [0.5, 0.5, 0, 0])
    assert_almost_equal(np.sum(probs), 1.0)
    print("✓ Test 6 passed: Measurement probabilities correct")

def test_stretch_goal_3_qubits():
    """Tests the n-qubit generalization for a 3-qubit (8 amplitude) case."""
    amps = [1, 0, 0, 0, 1j, 0, 0, 0] # Norm is sqrt(1+1) = sqrt(2)
    state = prepare_n_qubit_state(amps, num_qubits=3)
    assert_almost_equal(np.linalg.norm(state), 1.0)
    assert state.shape == (8,)
    val = 1 / np.sqrt(2)
    expected_state = [val, 0, 0, 0, 1j * val, 0, 0, 0]
    assert_allclose(state, expected_state)
    print("✓ Test 7 passed: 3-Qubit stretch goal (general)")

**Test Runner**

In [9]:
def run_all_tests():
    """Run all unit tests"""
    print("="*50)
    print("Running All Unit Tests")
    print("="*50)

    try:
        test_normalization_is_enforced()
        test_already_normalized_vector()
        test_output_dimension_is_correct()
        test_invalid_dimension_raises_error()
        test_zero_vector_raises_error()
        test_measurement_probabilities()
        test_stretch_goal_3_qubits()

        print("="*50)
        print("✅ ALL TESTS PASSED!")
        print("="*50)

    except AssertionError as e:
        print("\n" + "="*50)
        print(f"❌ TEST FAILED: {e}")
        print("="*50)

# --- Run the tests ---
run_all_tests()

Running All Unit Tests
✓ Test 1 passed: Normalization enforced
✓ Test 2 passed: Already normalized vector handled
✓ Test 3 passed: Correct output dimension (4,)
✓ Test 4 passed: Error raised for invalid dimension
✓ Test 5 passed: Zero vector error handling
✓ Test 6 passed: Measurement probabilities correct
✓ Test 7 passed: 3-Qubit stretch goal (general)
✅ ALL TESTS PASSED!


**Demonstration**

In [10]:
def demonstrate_usage():
    """Demonstrate the state preparation routine"""
    print("\n" + "="*50)
    print("DEMONSTRATION: State Preparation")
    print("="*50)

    # Example 1: Bell state
    print("\nExample 1: Bell state (|00⟩ + |11⟩)/√2")
    state1 = prepare_two_qubit_state([1, 0, 0, 1])
    print(f"  Input (unnormalized): [1, 0, 0, 1]")
    print(f"  State vector: {state1}")
    print(f"  Probabilities: {compute_measurement_probabilities(state1)}")

    # Example 2: Complex superposition
    print("\nExample 2: Complex superposition")
    state2 = prepare_two_qubit_state([1, 1j, -1, 1+1j])
    print(f"  Input (unnormalized): [1, 1j, -1, 1+1j]")
    print(f"  State vector: {state2}")
    print(f"  Probabilities: {compute_measurement_probabilities(state2)}")

    # Example 3: Three-qubit GHZ state (Stretch Goal)
    print("\nExample 3: Three-qubit GHZ state (|000⟩ + |111⟩)/√2")
    state3 = prepare_n_qubit_state([1, 0, 0, 0, 0, 0, 0, 1], num_qubits=3)
    print(f"  Input (unnormalized): [1, 0, 0, 0, 0, 0, 0, 1]")
    print(f"  State vector: {state3}")
    print(f"  Probabilities: {compute_measurement_probabilities(state3)}")

# --- Run the demonstration ---
demonstrate_usage()


DEMONSTRATION: State Preparation

Example 1: Bell state (|00⟩ + |11⟩)/√2
  Input (unnormalized): [1, 0, 0, 1]
  State vector: [0.70710678+0.j 0.        +0.j 0.        +0.j 0.70710678+0.j]
  Probabilities: [0.5 0.  0.  0.5]

Example 2: Complex superposition
  Input (unnormalized): [1, 1j, -1, 1+1j]
  State vector: [ 0.4472136+0.j         0.       +0.4472136j -0.4472136+0.j
  0.4472136+0.4472136j]
  Probabilities: [0.2 0.2 0.2 0.4]

Example 3: Three-qubit GHZ state (|000⟩ + |111⟩)/√2
  Input (unnormalized): [1, 0, 0, 0, 0, 0, 0, 1]
  State vector: [0.70710678+0.j 0.        +0.j 0.        +0.j 0.        +0.j
 0.        +0.j 0.        +0.j 0.        +0.j 0.70710678+0.j]
  Probabilities: [0.5 0.  0.  0.  0.  0.  0.  0.5]
